In [1]:
import requests
from utils import DayCET
import json
import os
import pandas as pd
from tqdm import tqdm
import time

## Token

In [2]:
def get_token_and_type(id):
    url = "https://digital.iservices.rte-france.com/token/oauth/"
    headers = {"Authorization": f"Basic {id}"}
    response = requests.post(url, headers=headers)
    if response.status_code == 200:
        resp_dict = response.json()
        return resp_dict['access_token'], resp_dict['token_type']
    else:
        print(f"Error: {response.status_code}, {response.text}")

## Consumption

In [3]:
rte_id_64 = "YWIwZDk0OWItOGM2NC00NTdiLWJlZTMtODU1NjRiMmEyZDc4OjY4ZWE4MTIyLTdmNjMtNDQzYi1iZDNiLTQ2YmVjOTFiZTEyMQ=="
ROOT_PATH = r'C:\Users\Raphaël Thireau\OneDrive - versoenergy\Documents\datapipeline'

In [4]:
def get_rte_conso(day : DayCET, RTE_token = None, id = rte_id_64):
    if RTE_token is None:
        token, token_type = get_token_and_type(id=id)
    else:
        token, token_type = RTE_token
    start = day.start_utc.tz_localize('UTC').isoformat().replace('+', '%2B')
    end = day.end_utc.tz_localize('UTC').isoformat().replace('+', '%2B')
    url = f'https://digital.iservices.rte-france.com/open_api/consumption/v1/short_term?start_date={start}&end_date={end}'
    headers = {"Authorization": f"{token_type} {token}"}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        with open(os.path.join(ROOT_PATH, 'process', 'data', 'RTE', 'Consumption', 'FR', f'{day.date_str.replace('-', '')}_RTE_Cons_FR.json'), 'w') as file:
            json.dump(response.json(), file, indent=4)
    else:
        raise Exception(f"Error: {response.status_code}, {response.text}")

In [5]:
RTE_tocken = get_token_and_type(rte_id_64)
for date in tqdm(pd.date_range(start='2024-04-13', end='2024-04-13', freq='D', inclusive='both')):
    dict_conso = get_rte_conso(day=DayCET(date.strftime(format='%Y-%m-%d')), RTE_token=RTE_tocken)

100%|██████████| 1/1 [00:03<00:00,  3.82s/it]


## Production

In [20]:
rte_id_64 = "YWIwZDk0OWItOGM2NC00NTdiLWJlZTMtODU1NjRiMmEyZDc4OjY4ZWE4MTIyLTdmNjMtNDQzYi1iZDNiLTQ2YmVjOTFiZTEyMQ=="
ROOT_PATH = r'C:\Users\Raphaël Thireau\OneDrive - versoenergy\Documents\datapipeline'

In [21]:
def get_rte_prod(day : DayCET, RTE_token = None, id = rte_id_64):
    if RTE_token is None:
        token, token_type = get_token_and_type(id=id)
    else:
        token, token_type = RTE_token
    start = day.start_utc.tz_localize('UTC').isoformat().replace('+', '%2B')
    end = day.end_utc.tz_localize('UTC').isoformat().replace('+', '%2B')
    url = f'https://digital.iservices.rte-france.com/open_api/actual_generation/v1/actual_generations_per_production_type?start_date={start}&end_date={end}'
    headers = {"Authorization": f"{token_type} {token}"}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        with open(os.path.join(ROOT_PATH, 'process', 'data', 'RTE', 'Production', 'FR', f'{day.date_str.replace('-', '')}_RTE_Prod_FR.json'), 'w') as file:
            json.dump(response.json(), file, indent=4)
    else:
        raise Exception(f"Error: {response.status_code}, {response.text}")

In [23]:
RTE_tocken = get_token_and_type(rte_id_64)
for date in tqdm(pd.date_range(start='2025-02-03', end='2025-03-02', freq='D', inclusive='both')):
    get_rte_prod(day=DayCET(date.strftime(format='%Y-%m-%d')), RTE_token=RTE_tocken)

  4%|▎         | 1/28 [00:00<00:05,  4.84it/s]

100%|██████████| 28/28 [00:04<00:00,  5.69it/s]


## Installed capacities

In [4]:
rte_id_64 = "OGM1NTdiNmItM2MwZS00NzFmLWIyNTItZTg1MjI3Y2NlZmE2Ojg2ODdiZWJhLThmMDctNGUyMi04MDU3LThlYjE4NmYyZjQ0NA=="
ROOT_PATH = r'C:\Users\Raphaël Thireau\OneDrive - versoenergy\Documents\datapipeline'

In [28]:
def get_rte_capa(year, RTE_token = None, id = rte_id_64):
    if RTE_token is None:
        token, token_type = get_token_and_type(id=id)
    else:
        token, token_type = RTE_token
    start = pd.Timestamp(year=year, month=1, day=1, tz='CET').isoformat().replace('+', '%2B')
    end = pd.Timestamp(year=year+1, month=1, day=1, tz='CET').isoformat().replace('+', '%2B')
    
    # url = f'https://digital.iservices.rte-france.com/open_api/generation_installed_capacities/v1/capacities_per_production_type?start_date={start}&end_date={end}'
    url = f'https://digital.iservices.rte-france.com/open_api/generation_installed_capacities/v1/capacities_cpc?start_date={start}&end_date={end}&departement_code=FR'
    headers = {"Authorization": f"{token_type} {token}"}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        # with open(os.path.join(ROOT_PATH, 'process', 'data', 'RTE', 'Capacities', 'FR', f'{year}_RTE_Capa_FR.json'), 'w') as file:
        #     json.dump(response.json(), file, indent=4)
        return response
    else:
        raise Exception(f"Error: {response.status_code}, {response.text}")

In [38]:
RTE_tocken = get_token_and_type(rte_id_64)
for year in range(2022, 2023):
    rep = get_rte_capa(year=year, RTE_token=RTE_tocken)

## Generation forecast

In [5]:
rte_id_64 = "YWIwZDk0OWItOGM2NC00NTdiLWJlZTMtODU1NjRiMmEyZDc4OjY4ZWE4MTIyLTdmNjMtNDQzYi1iZDNiLTQ2YmVjOTFiZTEyMQ=="
ROOT_PATH = r'C:\Users\Raphaël Thireau\OneDrive - versoenergy\Documents\datapipeline'

In [6]:
def get_rte_generation_forecast(day : DayCET, RTE_token = None, id = rte_id_64):
    if RTE_token is None:
        token, token_type = get_token_and_type(id=id)
    else:
        token, token_type = RTE_token
    start = day.start_utc.tz_localize('UTC').isoformat().replace('+', '%2B')
    end = day.end_utc.tz_localize('UTC').isoformat().replace('+', '%2B')
    url = f'https://digital.iservices.rte-france.com/open_api/generation_forecast/v2/forecasts?start_date={start}&end_date={end}&type=D-1&production_type=WIND_ONSHORE,WIND_OFFSHORE,SOLAR'
    headers = {"Authorization": f"{token_type} {token}"}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        with open(os.path.join(ROOT_PATH, 'process', 'data', 'RTE', 'GenerationForecast', 'FR', f'{day.date_str.replace('-', '')}_RTE_GenerationForecast_FR.json'), 'w') as file:
            json.dump(response.json(), file, indent=4)
    else:
        raise Exception(f"Error: {response.status_code}, {response.text}")

In [37]:
RTE_tocken = get_token_and_type(rte_id_64)
for date in tqdm(pd.date_range(start='2015-01-01', end='2025-02-02', freq='D', inclusive='both')):
    get_rte_generation_forecast(day=DayCET(date.strftime(format='%Y-%m-%d')), RTE_token=RTE_tocken)

100%|██████████| 3686/3686 [07:26<00:00,  8.25it/s]


## Imbalance

In [3]:
rte_id_64 = "YWIwZDk0OWItOGM2NC00NTdiLWJlZTMtODU1NjRiMmEyZDc4OjY4ZWE4MTIyLTdmNjMtNDQzYi1iZDNiLTQ2YmVjOTFiZTEyMQ=="
ROOT_PATH = r'C:\Users\Raphaël Thireau\OneDrive - versoenergy\Documents\datapipeline'

In [4]:
MAX_RETRIES = 5
RETRY_DELAY = 5

In [6]:
def get_rte_imbalance(day : DayCET, RTE_token = None, id = rte_id_64):
    if RTE_token is None:
        token, token_type = get_token_and_type(id=id)
    else:
        token, token_type = RTE_token
    start = day.start_utc.tz_localize('UTC').isoformat().replace('+', '%2B')
    end = day.end_utc.tz_localize('UTC').isoformat().replace('+', '%2B')
    url = f'https://digital.iservices.rte-france.com/open_api/balancing_energy/v3/imbalance_data?start_date={start}&end_date={end}'
    headers = {"Authorization": f"{token_type} {token}"}

    for attempt in range(MAX_RETRIES):

        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            with open(os.path.join(ROOT_PATH, 'process', 'data', 'RTE', 'Imbalance', 'FR', f'{day.date_str.replace('-', '')}_RTE_Imbalance_FR.json'), 'w') as file:
                json.dump(response.json(), file, indent=4)
            return  
        elif response.status_code == 500:
            time.sleep(RETRY_DELAY)
        else:
            raise Exception(f"Error: {response.status_code}, {response.text}")
    raise Exception()

In [7]:
for date in tqdm(pd.date_range(start='2025-02-10', end='2025-03-02', freq='D', inclusive='both')):
    RTE_tocken = get_token_and_type(rte_id_64)
    get_rte_imbalance(day=DayCET(date.strftime(format='%Y-%m-%d')), RTE_token=RTE_tocken)

100%|██████████| 21/21 [00:05<00:00,  3.74it/s]


## Balancing Capacity Market

In [4]:
rte_id_64 = "OGM1NTdiNmItM2MwZS00NzFmLWIyNTItZTg1MjI3Y2NlZmE2Ojg2ODdiZWJhLThmMDctNGUyMi04MDU3LThlYjE4NmYyZjQ0NA=="
ROOT_PATH = r'C:\Users\Raphaël Thireau\OneDrive - versoenergy\Documents\datapipeline'

In [5]:
def get_rte_balancing_capa(day : DayCET, RTE_token = None, id = rte_id_64):
    if RTE_token is None:
        token, token_type = get_token_and_type(id=id)
    else:
        token, token_type = RTE_token
    start = day.start_utc.tz_localize('UTC').isoformat().replace('+', '%2B')
    end = day.end_utc.tz_localize('UTC').isoformat().replace('+', '%2B')
    url = f'https://digital.iservices.rte-france.com/open_api/balancing_capacity/v4/procured_reserves?start_date={start}&end_date={end}'
    headers = {"Authorization": f"{token_type} {token}"}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        with open(os.path.join(ROOT_PATH, 'process', 'data', 'RTE', 'BalancingCapacity', 'FR', f'{day.date_str.replace('-', '')}_RTE_BalancingCapacity_FR.json'), 'w') as file:
            json.dump(response.json(), file, indent=4)
    else:
        raise Exception(f"Error: {response.status_code}, {response.text}")

In [11]:
for date in tqdm(pd.date_range(start='2025-02-03', end='2025-03-02', freq='D', inclusive='both')):
    RTE_tocken = get_token_and_type(rte_id_64)
    get_rte_balancing_capa(day=DayCET(date.strftime(format='%Y-%m-%d')), RTE_token=RTE_tocken)

100%|██████████| 28/28 [00:14<00:00,  1.90it/s]
